In [13]:
import re, glob, os
import numpy as np
os.chdir('/Users/cd8607/Documents/Lammps_MD')

In [14]:
files = glob.glob('sendover 2/*processed')
data = []

for _file in files:
    filename = os.path.basename(_file)

    if filename.startswith('He'):
        continue
    with open(_file, 'r') as file:
        lines = file.readlines()
    
    v  = int(filename[1])
    h  = int(filename[3])
    he = int(filename[6])
    
    ef = float(lines[41])
    rvol = float(lines[-1].split(' ')[3])
    
    rvol = 2 * rvol / 3.144221**3
    data.append([v, h, he, ef, rvol])

data = np.array(data)
sorted_indices = np.lexsort((data[:, 2], data[:, 1], data[:, 0]))
data = data[sorted_indices]
np.savetxt('dft_max.txt', data)

In [15]:
dft = np.loadtxt('dft_data_new.txt')
dft_new = []

for _row in dft:
    v, h, he, ef, img, rvol = _row
    idx = np.where((data[:, 0] == v) & (data[:, 1] == h) & (data[:, 2] == he))[0]

    if len(idx) == 1:
        if  not (v == 0 and h == 0 and he == 1):
            if np.isnan(ef):
                ef = data[idx, 3][0]
                print(ef)
            if np.isnan(rvol):
                rvol = data[idx, 4][0]
                print(rvol)
    _data = [v, h, he, ef, img, rvol]

    dft_new.append(_data)

dft_new = np.array(dft_new)
sorted_indices = np.lexsort((dft_new[:, 2], dft_new[:, 1], dft_new[:, 0]))
dft_new = dft_new[sorted_indices]
np.savetxt('dft_update.txt', dft_new, fmt='%.4f')

0.58681016004466
1.0449067032462862
1.508773746226639
1.9557352106548431
0.7649655798325934
1.1996583300492978
1.7673963907883092
2.1233023204509416
0.9601180058202121
1.3813775186685762
0.013665962289344573
2.342274119639561
1.1526854105448405
1.5373315201237843
2.0890122352273583
2.481025346528636
-0.11164416490297759
0.08723489243309857
0.37108640104214247
0.010082537636968496
0.2696614659715303
0.5193631757596622
0.09608428001808497
0.4204138632681317
0.6903391665173838
0.3022760890104503
0.5707746042908227
0.845224003627495
-0.4291316130117006
-0.2794349017829178
-0.06554991675808482
-0.36848307178584955
0.013665962289344573
0.06734769430904806
-0.2752583206113216
0.01899954124630076
0.2098291467538543
-0.16590130704348482
0.08498413244660896
0.335713560877207


In [42]:
files = glob.glob('sendover 2/*POSCAR')

if not os.path.exists('Fitting_Files'):
    os.mkdir('Fitting_Files')

for _file in files:

    dft_alat = 12.7400850036974127/4
    eam_alat = 3.144221

    filename = os.path.basename(_file)

    if filename.startswith('He'):
        continue
    
    v  = int(filename[1])
    h  = int(filename[3])
    he = int(filename[6])

    xyz = np.loadtxt(_file)
    
    xyz *= 4
    _data = []

    idx = 2*4**3 - v

    for i in range(h):
        _data.append([2, (xyz[idx, 0]), (xyz[idx, 1]), (xyz[idx, 2])])
        idx += 1

    for i in range(he):
        _data.append([3, (xyz[idx, 0]), (xyz[idx, 1]), (xyz[idx, 2])])
        idx += 1
    
    
    _data = np.array(_data)

    np.savetxt('Fitting_Files/V%dH%dHe%d.3.txt' % (v, h, he), _data)


In [41]:
files = glob.glob('EAM_Fit_Files/Atom_Files/*.atom')

if not os.path.exists('Fitting_Files'):
    os.mkdir('Fitting_Files')

for _file in files:

    dft_alat = 12.7400850036974127/4
    eam_alat = 3.144221

    filename = os.path.basename(_file)

    if filename.startswith('He'):
        continue
    
    v  = int(filename[1])
    h  = int(filename[3])
    he = int(filename[6])

    xyz = np.loadtxt(_file, skiprows=9)
    
    xyz[:, -3:] /= eam_alat

    _data = []

    for i in range(h + he):
        _data.append(xyz[-1 - i, 1:])

    _data = np.array(_data)

    np.savetxt('Fitting_Files/V%dH%dHe%d.2.txt' % (v, h, he), _data)


In [36]:
for file in glob.glob('Fitting_Files/*.txt'):
    
    if os.path.getsize(file) == 0:
        continue

    data = np.loadtxt(file)
    
    if data.ndim == 1:
        data = data.reshape(1, -1)

    data[:,0] = np.round(data[:,0])

    # data[:, 1:] /= eam_alat

    np.savetxt(file, data)

In [43]:
defect_centre = 2*np.ones((3,))

pos_arr = defect_centre + np.array([[0.25, 0.5, 0], [0.375, 0.375, 0], [0.5, 0.5, 0], [0.25, 0.25, 0.25]])

for i in range(4):

    file = 'Fitting_Files/V0H0He1.%d.txt' % i 

    xyz = np.loadtxt(file)

    xyz = xyz.reshape(1, -1)

    print(file, pos_arr[i])

    xyz[0, 1:] = pos_arr[i]

    np.savetxt(file, xyz)

Fitting_Files/V0H0He1.0.txt [2.25 2.5  2.  ]
Fitting_Files/V0H0He1.1.txt [2.375 2.375 2.   ]
Fitting_Files/V0H0He1.2.txt [2.5 2.5 2. ]
Fitting_Files/V0H0He1.3.txt [2.25 2.25 2.25]


In [48]:
for i in range(4):
    process = 'sendover 2/He_vasp_%s.processed' % (i)
    poscar = 'sendover 2/He_vasp_%s.POSCAR' % (i)

    
    xyz = np.loadtxt(_file)
    
    xyz *= 4
    _data = []

    idx = 2*4**3 - v

    for i in range(2):
        _data.append([3, (xyz[idx, 0]), (xyz[idx, 1]), (xyz[idx, 2])])
        idx += 1
    
    _data = np.array(_data)

    with open(process, 'r') as file:
        lines = file.readlines()
    
    v  = int(filename[1])
    h  = int(filename[3])
    he = int(filename[6])
    
    ef = float(lines[41])
    rvol = float(lines[-1].split(' ')[3])
    
    rvol = 2 * rvol / 3.144221**3   
    
    print(ef, rvol)

    file = 'Fitting_Files/V0H0He2.%d.txt' % i 
    
    np.savetxt(file, _data)

11.457984998127941 0.8523716463871255
11.728034575700704 0.8480844310228706
12.515645536225648 0.7245089581162585
12.384293764731083 0.7459671617760658
